In [11]:
from source import create_df, preprocess, distributions, popularity_recommendations
import pandas as pd
import sqlite3
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

import ipywidgets as ipyw               
from ipywidgets import widgets, interactive_output, interact, interactive, fixed, widget
from IPython.display import display

# **EDA**

In [13]:
%reload_ext sql
%sql sqlite:///db_movies

In [ ]:
%%sql

SELECT movieId, COUNT(*) AS total_ratings FROM ratings
GROUP BY movieId
HAVING total_ratings <= 1;


SELECT userId, COUNT(*) AS watched_movies FROM ratings
GROUP BY userId
ORDER BY watched_movies DESC;

SELECT title, movies.movieId FROM movies
INNER JOIN (SELECT movieId, COUNT(*) AS total_ratings FROM ratings
GROUP BY movieId
HAVING total_ratings <= 1) t1
ON movies.movieId = t1.movieId
ORDER BY title;


SELECT * FROM movies
WHERE genres = "(no genres listed)";

SELECT * FROM (SELECT movieId, COUNT(*) AS total_ratings FROM ratings
GROUP BY movieId
ORDER BY total_ratings) t1
INNER JOIN (SELECT * FROM movies
WHERE genres = "(no genres listed)") t2
ON t1.movieId = t2.movieId;

SELECT movieId, COUNT(*) AS total_ratings FROM ratings
GROUP BY movieId
ORDER BY total_ratings;


SELECT userId, COUNT(*) AS watched_movies FROM ratings
GROUP BY userId
ORDER BY watched_movies DESC;


SELECT userId FROM (SELECT userId, COUNT(*) AS watched_movies FROM ratings
GROUP BY userId
HAVING watched_movies > 1000);


SELECT rel_yr, title, ROUND(AVG(rating), 3) AS avg_ratings, COUNT(*) total_ratings FROM full_info
GROUP BY rel_yr, title
HAVING rel_yr = 2018
ORDER BY rel_yr DESC, avg_ratings DESC;




In [ ]:
distributions()

In [ ]:
preprocess()

In [ ]:
distributions()

In [ ]:
query = "SELECT userId, COUNT(*) AS watched_movies FROM ratings GROUP BY userId ORDER BY watched_movies DESC"
df = create_df(query)

fig = px.histogram(df, x="watched_movies", nbins=60, template="plotly_white", labels={"watched_movies":"<b>Peliculas vistas</b>"},
                  title="<b>Distribucion Cantidad de peliculas vistas<br>por usuario</b>", color_discrete_sequence=["rgba(109, 176, 223, 0.8)"])

fig.update_layout(title={"x":0.5})

fig.update_traces(hovertemplate="<b>Peliculas vistas</b>=%{x}<br><b>Usuarios</b>=%{y}<extra></extra>")

fig.update_yaxes(title="<b>Usuarios</b>")

fig.show()

In [ ]:
query = "SELECT movieId, COUNT(*) AS total_ratings FROM ratings GROUP BY movieId ORDER BY total_ratings"
df = create_df(query)
# df["conteo"] = df["conteo"]/df["conteo"].sum() 

fig = px.histogram(df, x="total_ratings", nbins=50, template="plotly_white", labels={"total_ratings":"<b>Veces Calificada</b>"},
                  title="<b>Distribucion Cantidad de calificaciones por pelicula</b>", color_discrete_sequence=["rgba(105, 153, 66, 0.8)"])

fig.update_layout(title={"x":0.5})

fig.update_traces(textposition="outside", hovertemplate="<b>Veces Calificada</b>=%{x}<br><b>Peliculas</b>=%{y}<extra></extra>")

fig.update_yaxes(title="<b>Peliculas</b>", )

fig.show()

In [2]:
 popularity_recommendations()

Output()

In [12]:
title = "No Game No Life: Zero (2017)"
n = 11

query = "SELECT * FROM movies;"

movies = create_df(query).set_index("movieId")

movies["release_yr"] = movies["title"].str.extract("\((\d{4})\)")

escaler = MinMaxScaler()

index = movies[movies["title"] ==  title].index.values.tolist()[0]

to_corr = pd.concat([movies["release_yr"], movies['genres'].str.get_dummies()], axis=1)

to_corr = pd.DataFrame(escaler.fit_transform(to_corr), columns=to_corr.columns, index=to_corr.index)


movies["similitud"] = to_corr.corrwith(to_corr.loc[index], axis=1)


display(movies.sort_values(by="similitud", ascending=False).iloc[:n,:4])

,title,genres,release_yr,similitud
movieId,,,,
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,2017,1.000000
139855,Anomalisa (2015),Animation|Comedy|Fantasy,2015,0.999965
126577,"Daddy, I'm A Zombie (2012)",Animation|Comedy|Fantasy,2012,0.999778
79008,South Park: Imaginationland (2008),Animation|Comedy|Fantasy,2008,0.999270
6773,"Triplets of Belleville, The (Les triplettes de...",Animation|Comedy|Fantasy,2003,0.998197
4158,Monkeybone (2001),Animation|Comedy|Fantasy,2001,0.997626
72692,Mickey's Once Upon a Christmas (1999),Animation|Comedy|Fantasy,1999,0.996971
7228,Cool World (1992),Animation|Comedy|Fantasy,1992,0.993998
182297,Porky in Wackyland (1938),Animation|Comedy|Fantasy,1938,0.929678


In [14]:
%reload_ext sql
%sql sqlite:///db_movies
conn=sqlite3.connect('db_movies')
cur=conn.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('ratings',), ('movies',)]

In [16]:
df_movies=pd.read_sql("select * from movies", conn)
df_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [20]:
from source import create_df, preprocess, distributions
import sqlite3
import plotly.express as px

In [21]:
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib

In [26]:
df = pd.read_sql("SELECT * FROM full_info", conn)
df

,userId,movieId,title,genres,rating,timestamp,rel_yr
0,1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,2000-07-30 18:45:03,1995
1,1,3,Grumpier Old Men (1995),Comedy|Romance,4.0,2000-07-30 18:20:47,1995
2,1,6,Heat (1995),Action|Crime|Thriller,4.0,2000-07-30 18:37:04,1995
3,1,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,5.0,2000-07-30 19:03:35,1995
4,1,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,5.0,2000-07-30 18:48:51,1995
...,...,...,...,...,...,...,...
82314,609,892,Twelfth Night (1996),Comedy|Drama|Romance,3.0,1996-11-05 19:11:20,1996
82315,609,1056,Jude (1996),Drama,3.0,1996-11-05 19:11:20,1996
82316,609,1059,William Shakespeare's Romeo + Juliet (1996),Drama|Romance,3.0,1996-11-05 19:10:54,1996
82317,609,1150,"Return of Martin Guerre, The (Retour de Martin...",Drama,4.0,1996-11-05 19:10:54,1982


In [27]:
create_df("SELECT * FROM full_info")

,userId,movieId,title,genres,rating,timestamp,rel_yr
0,1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,2000-07-30 18:45:03,1995
1,1,3,Grumpier Old Men (1995),Comedy|Romance,4.0,2000-07-30 18:20:47,1995
2,1,6,Heat (1995),Action|Crime|Thriller,4.0,2000-07-30 18:37:04,1995
3,1,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,5.0,2000-07-30 19:03:35,1995
4,1,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,5.0,2000-07-30 18:48:51,1995
...,...,...,...,...,...,...,...
82314,609,892,Twelfth Night (1996),Comedy|Drama|Romance,3.0,1996-11-05 19:11:20,1996
82315,609,1056,Jude (1996),Drama,3.0,1996-11-05 19:11:20,1996
82316,609,1059,William Shakespeare's Romeo + Juliet (1996),Drama|Romance,3.0,1996-11-05 19:10:54,1996
82317,609,1150,"Return of Martin Guerre, The (Retour de Martin...",Drama,4.0,1996-11-05 19:10:54,1982


In [28]:
preprocess()

In [29]:
df = pd.read_sql("SELECT * FROM full_info", conn)
df

,userId,movieId,title,genres,rating,timestamp,rel_yr
0,1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,2000-07-30 18:45:03,1995
1,1,3,Grumpier Old Men (1995),Comedy|Romance,4.0,2000-07-30 18:20:47,1995
2,1,6,Heat (1995),Action|Crime|Thriller,4.0,2000-07-30 18:37:04,1995
3,1,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,5.0,2000-07-30 19:03:35,1995
4,1,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,5.0,2000-07-30 18:48:51,1995
...,...,...,...,...,...,...,...
82314,609,892,Twelfth Night (1996),Comedy|Drama|Romance,3.0,1996-11-05 19:11:20,1996
82315,609,1056,Jude (1996),Drama,3.0,1996-11-05 19:11:20,1996
82316,609,1059,William Shakespeare's Romeo + Juliet (1996),Drama|Romance,3.0,1996-11-05 19:10:54,1996
82317,609,1150,"Return of Martin Guerre, The (Retour de Martin...",Drama,4.0,1996-11-05 19:10:54,1982


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82319 entries, 0 to 82318
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     82319 non-null  int64  
 1   movieId    82319 non-null  int64  
 2   title      82319 non-null  object 
 3   genres     82319 non-null  object 
 4   rating     82319 non-null  float64
 5   timestamp  82319 non-null  object 
 6   rel_yr     82305 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 4.4+ MB


In [46]:
df['rel_yr'].isnull().sum()
# df.info()

14

In [49]:
df[df['rel_yr'].isnull()]

,userId,movieId,title,genres,rating,timestamp,rel_yr
9147,62,176601,Black Mirror,(no genres listed),5.0,2018-05-08 16:00:52,None
15626,105,147250,The Adventures of Sherlock Holmes and Doctor W...,(no genres listed),5.0,2018-05-13 10:29:14,None
15653,105,171749,Death Note: Desu nôto (2006–2007),(no genres listed),5.0,2018-05-13 10:29:25,None
16619,111,171631,Maria Bamford: Old Baby,(no genres listed),1.0,2018-01-31 23:21:49,None
16621,111,171891,Generation Iron 2,(no genres listed),3.5,2018-01-31 23:09:59,None
28779,209,140956,Ready Player One,Action|Sci-Fi|Thriller,3.5,2018-04-23 22:23:59,None
28830,210,40697,Babylon 5,Sci-Fi,4.0,2018-05-25 16:36:31,None
41886,305,140956,Ready Player One,Action|Sci-Fi|Thriller,4.5,2018-07-31 13:26:17,None
42004,306,149334,Nocturnal Animals,Drama|Thriller,3.0,2018-02-11 20:19:32,None
45144,318,171495,Cosmos,(no genres listed),5.0,2018-01-24 20:40:06,None


In [57]:
df[df['title'] == 'Black Mirror']


,userId,movieId,title,genres,rating,timestamp,rel_yr
9147,62,176601,Black Mirror,(no genres listed),5.0,2018-05-08 16:00:52,None
